### Riddler Express:

The following 8-by-8 grid is covered with a total of 64 chess pieces, with one piece on each square. You should begin this puzzle at the white bishop on the green square. You can then move from white piece to white piece via the following rules:

- If you are on a pawn, move up one space diagonally (left or right).
- If you are on a knight, move in an “L” shape — two spaces up, down, left or right, and then one space in a perpendicular direction.
- If you are on a bishop, move one space in any diagonal direction.
- If you are on a rook, move one space up, down, left or right.
- If you are on a queen, move one space in any direction (horizontal, vertical or diagonal).


### Photo Of Start:

### Coding it up:

- to simplify I will just call all `black` pieces `BL` since we need to avoid these. 
- Kings we land on are `K` -> hitting one indicates a win
- Since it is a fixed grid it isn't a big deal to just use a nested list, `numpy` would be better for speed but working with characters in `numpy` is a bit weirder and I just want to keep tabs based on characters.

### Dictionary of codes:
- K -> King
- BL -> black piece
- b -> bishop, diagonal
- r -> rook, horizontal or vertical
- n -> knight, 2/1 process
- p -> pawn, diagonal upwards
- q -> Queen, single move in any direction 

### Building Main Board:

Lists concern me typically but I won't be overriding anything for game board, just keeping track of starting conditions in a tuple

In [50]:
from itertools import product

game_board = [
     ['K', 'BL', 'b', 'BL', 'K', 'r', 'b', 'r'],
     ['n', 'r', 'n', 'n', 'p', 'n', 'K', 'b'],
     ['r', 'BL', 'n', 'BL', 'p', 'r', 'p', 'r'],
     ['BL', 'BL', 'p', 'r', 'BL', 'n', 'BL', 'BL'],
     ['n', 'n', 'n', 'b', 'BL', 'b', 'r', 'b'],
     ['q', 'r', 'n', 'p', 'BL', 'n', 'r', 'q'],
     ['BL', 'BL', 'r', 'p', 'b', 'p', 'b', 'q'],
     ['K', 'b', 'q', 'n', 'p', 'r', 'n', 'n']
]

# set boundaries for future ref
min_bound = 0
max_bound = 7

# start
start = (6,4)

### Building Movement Functions:

Lots of testing in here

In [52]:
def bishopMove(row, col):
    """Bishop can move diagonal 1 space"""
    return list(product([row-1, row+1],[col-1, col+1]))

# test starting point: we expect 4 likely positions of [7][3], [5][3], [5][5], [7][5]

output = bishopMove(6,4)
print(output)
test_board = [['_'] * 8 for i in range(8)]
test_board[6][4] = 'B'
for row, col in output:
    test_board[row][col] = 'X'
test_board

[(5, 3), (5, 5), (7, 3), (7, 5)]


[['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', 'X', '_', 'X', '_', '_'],
 ['_', '_', '_', '_', 'B', '_', '_', '_'],
 ['_', '_', '_', 'X', '_', 'X', '_', '_']]

In [53]:
def rookMove(row, col):
    """Rook can move horiz/vert 1 space"""
    moves = []
    for i,j in [(-1,0), (1,0),(0,-1), (0,1)]:
        moves.append((row + i, col + j))
    
    return moves

# test for rook in row 3, col 3 (using 0 index of course): [2][3], [4][3], [3][4], [3][2]
output = rookMove(3,3)
test_board = [['_'] * 8 for i in range(8)]
test_board[3][3] = 'R'
for row, col in output:
    test_board[row][col] = 'X'
test_board

[['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', 'X', '_', '_', '_', '_'],
 ['_', '_', 'X', 'R', 'X', '_', '_', '_'],
 ['_', '_', '_', 'X', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_']]

In [54]:
def knightMove(row, col):
    """Knight can move 2, 1 per usual (l-shape) -> 8 total moves
    Stack tips: https://stackoverflow.com/questions/19372622/how-do-i-generate-all-of-a-knights-moves
    """
    return list(product([row-1, row+1],[col-2, col+2])) + list(product([row-2,row+2],[col-1,col+1]))

# test for knight in row 4, col 4 (using 0 index of course): we should have 8 total moves, confirmed visually on board
output = knightMove(4,4)
test_board = [['_'] * 8 for i in range(8)]
test_board[4][4] = 'N'
for row, col in output:
    test_board[row][col] = 'X'
test_board

[['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', 'X', '_', 'X', '_', '_'],
 ['_', '_', 'X', '_', '_', '_', 'X', '_'],
 ['_', '_', '_', '_', 'N', '_', '_', '_'],
 ['_', '_', 'X', '_', '_', '_', 'X', '_'],
 ['_', '_', '_', 'X', '_', 'X', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_']]

In [55]:
def pawnMove(row, col):
    """Pawn can only move up-diagonal"""
    moves = []
    for i,j in [(-1,-1), (-1,1)]:
        moves.append((row + i, col + j))
    
    return moves

# test starting point: we expect 4 likely positions of [7][3], [5][3], [5][5], [7][5]
output = pawnMove(4,4)
test_board = [['_'] * 8 for i in range(8)]
test_board[4][4] = 'P'
for row, col in output:
    test_board[row][col] = 'X'
test_board

[['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', 'X', '_', 'X', '_', '_'],
 ['_', '_', '_', '_', 'P', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_']]

In [62]:
def queenMove(row, col):
    """Queen can move one space in any direction"""
    all_moves = list(product([row-1, row+1, row],[col-1, col+1, col]))
    
    return [move for move in all_moves if (move[0] != row or move[1] != col)] # need to ensure some move occurred

output = queenMove(4,4)
print(output)
test_board = [['_'] * 8 for i in range(8)]
test_board[4][4] = 'Q'
for row, col in output:
    test_board[row][col] = 'X'
test_board

[(3, 3), (3, 5), (3, 4), (5, 3), (5, 5), (5, 4), (4, 3), (4, 5)]


[['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', 'X', 'X', 'X', '_', '_'],
 ['_', '_', '_', 'X', 'Q', 'X', '_', '_'],
 ['_', '_', '_', 'X', 'X', 'X', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_']]

### Moving & Checking Boundaries

In [79]:
def move(p, row, col):
    """Determine current piece & return list of moves (not yet filtered)"""
    if p == 'b':
        return bishopMove(row, col)
    elif p == 'r':
        return rookMove(row, col)
    elif p == 'n':
        return knightMove(row, col)
    elif p == 'p':
        return pawnMove(row, col)
    elif p == 'q':
        return queenMove(row, col)
    else:
        raise TypeError("Not correct piece")

def checkMoveBoundary(moveList):
    """Check if any move goes outside boundary, if so pass"""
    return [(x,y) for x,y in moveList if x >= 0 and y >= 0 and x < 8 and y < 8]
    
def checkMoveBlack(moveList, gameBoard):
    """Check if any move leads to a black piece; remove as option"""
    return [(x,y) for x,y in moveList if gameBoard[x][y] != 'BL']
    
def checkMoveKing():
    """Check if we can move to a king (which results in a win)"""
    
    return False

In [80]:
# testing move output
assert(move('b', 6, 4) == [(5, 3), (5, 5), (7, 3), (7, 5)])
assert(move('q', 4,4) == [(3, 3), (3, 5), (3, 4), (5, 3), (5, 5), (5, 4), (4, 3), (4, 5)])

In [87]:
# check boundary for rook at bottom: should only have 3 moves available after passing through boundary
rook_moves = move('r', 7, 5)
print(rook_moves)
assert(checkMoveBoundary(rook_moves) == [(6, 5), (7, 4), (7, 6)])

[(6, 5), (8, 5), (7, 4), (7, 6)]


In [86]:
# check boundary for black 
# let's say we started on rook row 4, col 4. (even though this is a black rook, its a good test)
# we expect[(3, 4), (5, 4), (4, 3), (4, 5)]
# however, (3, 4) & (5, 4)are both BL on gameboard, so must be removed from consideration
rook_moves = move('r', 4, 4)
print(rook_moves)
assert(checkMoveBlack(rook_moves,game_board) == [(4, 3), (4, 5)])

[(3, 4), (5, 4), (4, 3), (4, 5)]
